In [2]:
import sys
import os

sys.path.append(os.path.join('..', '..', '..'))

In [7]:
from roboflow import Roboflow
from utils import get_settings

settings = get_settings()
ROBOFLOW_API_KEY = settings.ROBOFLOW_API_KEY
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("mihailo").project("football-players-detection-3zvbc-7ocfe")
version = project.version(2)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to football-players-detection-2 in yolov11:: 100%|██████████| 1948/1948 [00:01<00:00, 1408.97it/s]


In [8]:
import os
import yaml

YAML_PATH = os.path.join(dataset.location,"data.yaml")

with open(YAML_PATH) as f:
     list_doc = yaml.safe_load(f)

list_doc['test'] = '../test/images'
list_doc['train'] = '../train/images'
list_doc['val'] = '../valid/images'

print(list_doc)

with open(YAML_PATH, "w") as f:
    yaml.dump(list_doc, f)

{'train': '../train/images', 'val': '../valid/images', 'test': '../test/images', 'nc': 4, 'names': ['ball', 'goalkeeper', 'player', 'referee'], 'roboflow': {'workspace': 'mihailo', 'project': 'football-players-detection-3zvbc-7ocfe', 'version': 2, 'license': 'CC BY 4.0', 'url': 'https://universe.roboflow.com/mihailo/football-players-detection-3zvbc-7ocfe/dataset/2'}}


In [13]:
YAML_PATH

'd:\\Academic\\Code\\Projects\\football\\football-players-detection-2\\data.yaml'

In [ ]:
!yolo mode=train model=yolo11s.pt task=detect data={YAML_PATH} epochs=200 patience=20 imgsz=1280 batch=8 plots=True cache=False

In [9]:
RESULTS_DIR = 'runs/detect/train_yolov11'

In [10]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.figure(figsize=(12,12))
plt.axis(False)
plt.imshow(mpimg.imread(f'{RESULTS_DIR}/val_batch0_pred.jpg'))

plt.show()

In [19]:
!yolo task=detect mode=val model={RESULTS_DIR}/weights/best.pt data={YAML_PATH}

Ultralytics 8.3.9 🚀 Python-3.11.4 torch-2.4.1+cpu CPU (Intel Core(TM) i5-10300H 2.50GHz)
YOLO11s summary (fused): 238 layers, 9,414,348 parameters, 0 gradients, 21.3 GFLOPs
val: New cache created: D:\Academic\Code\Projects\football\football-players-detection-2\valid\labels.cache
                   all         49       1174      0.888      0.876      0.903      0.658
                  ball         45         45      0.781        0.6      0.667      0.329
            goalkeeper         38         39       0.88      0.941      0.965      0.773
                player         49        973      0.961      0.988      0.994      0.834
               referee         49        117      0.928      0.974      0.984      0.696
Speed: 14.6ms preprocess, 866.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\val3
💡 Learn more at https://docs.ultralytics.com/modes/val



val: Scanning D:\Academic\Code\Projects\football\football-players-detection-2\valid\labels...:   0%|          | 0/49 [00:00<?, ?it/s]
val: Scanning D:\Academic\Code\Projects\football\football-players-detection-2\valid\labels... 1 images, 0 backgrounds, 0 corrupt:   2%|▏         | 1/49 [00:00<00:08,  5.94it/s]
val: Scanning D:\Academic\Code\Projects\football\football-players-detection-2\valid\labels... 49 images, 0 backgrounds, 0 corrupt: 100%|██████████| 49/49 [00:00<00:00, 192.23it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/4 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  25%|██▌       | 1/4 [00:15<00:46, 15.52s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 2/4 [00:29<00:29, 14.79s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  75%|███████▌  | 

In [20]:
import os
import shutil

source_path = f'{RESULTS_DIR}/weights/best.pt'
destination_dir = '../../weights'
destination_file = os.path.join(destination_dir, 'object-detection.pt')

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Move and rename the file
try:
    shutil.move(source_path, destination_file)
    print(f"File moved and renamed to {destination_file}")
except FileNotFoundError:
    print(f"Source file '{source_path}' not found.")
except Exception as e:
    print(f"Error occurred: {e}")

File moved and renamed to ../../weights\object-detection.pt
